In [1]:
%reload_ext autoreload
%autoreload 2
from src.acnets.pipeline import Parcellation
from src.acnets.pipeline import TimeseriesAggregator, ConnectivityExtractor
from src.acnets.pipeline import ConnectivityAggregator
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline

In [16]:
# parcellation
parcellation = Parcellation(atlas_name='dosenbach2010')
dataset = parcellation.fit_transform(None)

# time-series aggregation
ts_agg = TimeseriesAggregator(strategy=None)
dataset = ts_agg.fit_transform(dataset)

# connectivity extraction
conn_extractor = ConnectivityExtractor(kind='correlation')
dataset = conn_extractor.fit_transform(dataset)

# connectivity aggregation
conn_agg = ConnectivityAggregator(strategy='network')
dataset = conn_agg.fit_transform(dataset)

In [ ]:
debug_pipe = Pipeline([
    ('parcellation', Parcellation('dosenbach2010')),
    ('timeseries_aggregation', TimeseriesAggregator(strategy=None)),
    ('connectivity', ConnectivityExtractor('partial correlation')),
    ('connectivity_aggregation', ConnectivityAggregator(strategy='network')),
])

debug_pipe.fit_transform(None)  # load data for all participants
